In [1]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

/home/kurianbenoy/mambaforge/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/kurianbenoy/mambaforge/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs
  warn(f"Failed to load image Python extension: {e}")


In [2]:
#hide
from fastbook import *
from IPython.display import display,HTML

# Data Munging with fastai's Mid-Level API

## Going Deeper into fastai's Layered API

In [3]:
from fastai.text.all import *

dls = TextDataLoaders.from_folder(untar_data(URLs.IMDB), valid='test')

In [4]:
path = untar_data(URLs.IMDB)
dls = DataBlock(
    blocks=(TextBlock.from_folder(path),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path)

In [5]:
dls.show_batch()

/home/kurianbenoy/mambaforge/lib/python3.9/site-packages/fastai/torch_core.py:476: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ax = ax.append(pd.Series({label: o}))
/home/kurianbenoy/mambaforge/lib/python3.9/site-packages/fastai/torch_core.py:476: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ax = ax.append(pd.Series({label: o}))
/home/kurianbenoy/mambaforge/lib/python3.9/site-packages/fastai/torch_core.py:476: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ax = ax.append(pd.Series({label: o}))
/home/kurianbenoy/mambaforge/lib/python3.9/site-packages/fastai/torch_core.py:476: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ax = ax.ap

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,"xxbos i thought that xxup rotj was clearly the best out of the three xxmaj star xxmaj wars movies . i find it surprising that xxup rotj is considered the weakest installment in the xxmaj trilogy by many who have voted . xxmaj to me it seemed like xxup rotj was the best because it had the most profound plot , the most suspense , surprises , most xxunk the ending ) and definitely the most episodic movie . i personally like the xxmaj empire xxmaj strikes xxmaj back a lot also but i think it is slightly less good than than xxup rotj since it was slower - moving , was not as episodic , and i just did not feel as much suspense or emotion as i did with the third movie . \n\n xxmaj it also seems like to me that after reading these surprising reviews that",pos
2,"xxbos xxmaj heavy - handed moralism . xxmaj writers using characters as mouthpieces to speak for themselves . xxmaj predictable , plodding plot points ( say that five times fast ) . a child 's imitation of xxmaj britney xxmaj spears . xxmaj this film has all the earmarks of a xxmaj lifetime xxmaj special reject . \n\n i honestly believe that xxmaj jesus xxmaj xxunk and xxmaj julia xxmaj xxunk set out to create a thought - provoking , emotional film on a tough subject , exploring the idea that things are not always black and white , that one who is a criminal by definition is not necessarily a bad human being , and that there can be extenuating circumstances , especially when one puts the well - being of a child first . xxmaj however , their earnestness ends up being channeled into preachy dialogue and trite",neg
3,"xxbos xxmaj jim xxmaj carrey is back to much the same role that he played in xxmaj the xxmaj mask , a timid guy who is trying to get ahead in the world but who seems to be plagued with bad luck . xxmaj even when he tries to help a homeless guy from being harassed by a bunch of hoodlums ( and of course they have to be xxmaj mexican , obviously ) , his good will towards his fellow man backfires . xxmaj in that case , it was n't too hard to predict that he was about to have a handful of angry hoodlums , but i like that the movie suggests that things like that should n't be ignored . xxmaj i 'm reminded of the episode of xxmaj michael xxmaj moore 's brilliant xxmaj the xxmaj awful xxmaj truth , when they had a man",pos
4,"xxbos xxmaj god ! xxmaj zorro has been the the subject of about as many movies as xxmaj tarzan , and probably had about as many actors in the title role . \n\n xxmaj this xxmaj serial is one of my own personal favourites , and as previously stated , it is one of the xxmaj top 5 xxmaj sound xxmaj serials . xxmaj oddly enough , this is one production that came out in that water shed year of 1939 . * xxmaj by the time of this production in ' 39 , xxmaj zorro was really well known as a ( pulp ) literary and movie character . xxmaj the film opens up with a little foot note about the xxmaj history of the xxmaj mexico 's struggle for freedom from rule by a xxmaj european xxmaj monarchy , namely xxmaj spain . xxmaj the story invites comparison",pos
5,"xxbos xxmaj to be a xxmaj buster xxmaj keaton fan is to have your heart broken on a reg

In [6]:
doc(TextBlock)

class TextBlock [source] TextBlock ( tok_tfm , vocab = None , is_lm = False , seq_len = 72 , backwards = False , min_freq = 3 , max_vocab = 60000 , special_toks = None ) :: TransformBlock 
 
 A TransformBlock for texts 
 Show in docs

In [7]:
Numericalize??

Init signature:
Numericalize(
    self,
    vocab=None,
    min_freq=3,
    max_vocab=60000,
    special_toks=None,
)
Source:        
class Numericalize(Transform):
    "Reversible transform of tokenized texts to numericalized ids"
    def __init__(self, vocab=None, min_freq=3, max_vocab=60000, special_toks=None):
        store_attr('vocab,min_freq,max_vocab,special_toks')
        self.o2i = None if vocab is None else defaultdict(int, {v:k for k,v in enumerate(vocab)})

    def setups(self, dsets):
        if dsets is None: return
        if self.vocab is None:
            count = dsets.counter if getattr(dsets, 'counter', None) is not None else Counter(p for o in dsets for p in o)
            if self.special_toks is None and hasattr(dsets, 'special_toks'):
                self.special_toks = dsets.special_toks
            self.vocab = make_vocab(count, min_freq=self.min_freq, max_vocab=self.max_vocab, special_toks=self.special_toks)
            self.o2i = defaultdict(int, {v:k for k,v

**What is hasattr? in python**

In [8]:
class Car:
    def __init__(self, model_name: str, brand: str, year: int):
        self.model = model_name
        self.brand = brand
        self.year = year

In [9]:
c = Car(model_name="Verna", brand="Hyndai", year=2009)

In [11]:
c.model

'Verna'

In [12]:
hasattr(c,"model")

True

In [13]:
hasattr(c, "price")

False

In [14]:
hasattr(c, "year")

True

### Transforms

In [15]:
files = get_text_files(path, folders = ['train', 'test'])
txts = L(o.open().read() for o in files[:2000])

In [16]:
txts

(#2000) ["What could have been an excellent hostage movie was totally ruined by what apparently looks like a bored director ... there were so many directions that the movie could have taken ... a vampire slash-fest was not one of these!!! The last 45 mins. or so results in the movie being an absolutely ridiculous waste of time. ...and sex machine?? ... you gotta be kidding me! The acting talents of the likes of Juliette Lewis and Harvey Keitel (not to mention George Clooney) are completely wasted in this nonsensical movie. <br /><br />The director... Robert Rodriguez, known for his other gory flicks including el mariachi, desperado, once upon a time in Mexico, and the very recent sin city ... really holds your attention with the well executed first half ... which leads you to believe that you are in for an entertaining time ... but then apparently for no reason, and without any provocation, the madness starts ... there's even feeble attempts at parody and comedy ... truly exasperating!

In [18]:
Tokenizer??

Init signature:
Tokenizer(
    self,
    tok,
    rules=None,
    counter=None,
    lengths=None,
    mode=None,
    sep=' ',
)
Source:        
class Tokenizer(Transform):
    "Provides a consistent `Transform` interface to tokenizers operating on `DataFrame`s and folders"
    input_types = (str, list, L, tuple, Path)
    def __init__(self, tok, rules=None, counter=None, lengths=None, mode=None, sep=' '):
        if isinstance(tok,type): tok=tok()
        store_attr('tok,counter,lengths,mode,sep')
        self.rules = defaults.text_proc_rules if rules is None else rules

    @classmethod
    @delegates(tokenize_df, keep=True)
    def from_df(cls, text_cols, tok=None, rules=None, sep=' ', **kwargs):
        if tok is None: tok = WordTokenizer()
        res = cls(tok, rules=rules, mode='df')
        res.kwargs,res.train_setup = merge({'tok': tok}, kwargs),False
        res.text_cols,res.sep = text_cols,sep
        return res

    @classmethod
    @delegates(tokenize_folder, keep=True)
  

In [19]:
tok = Tokenizer.from_folder(path)
tok.setup(txts)
toks = txts.map(tok)
toks[0]

(#207) ['xxbos','xxmaj','what','could','have','been','an','excellent','hostage','movie'...]

In [25]:
toks

(#2000) [['xxbos', 'xxmaj', 'what', 'could', 'have', 'been', 'an', 'excellent', 'hostage', 'movie', 'was', 'totally', 'ruined', 'by', 'what', 'apparently', 'looks', 'like', 'a', 'bored', 'director', '…', 'there', 'were', 'so', 'many', 'directions', 'that', 'the', 'movie', 'could', 'have', 'taken', '…', 'a', 'vampire', 'slash', '-', 'fest', 'was', 'not', 'one', 'of', 'these', 'xxrep', '3', '!', 'xxmaj', 'the', 'last', '45', 'mins', '.', 'or', 'so', 'results', 'in', 'the', 'movie', 'being', 'an', 'absolutely', 'ridiculous', 'waste', 'of', 'time', '.', '…', 'and', 'sex', 'machine', '?', '?', '…', 'you', 'got', 'ta', 'be', 'kidding', 'me', '!', 'xxmaj', 'the', 'acting', 'talents', 'of', 'the', 'likes', 'of', 'xxmaj', 'juliette', 'xxmaj', 'lewis', 'and', 'xxmaj', 'harvey', 'xxmaj', 'keitel', '(', 'not', 'to', 'mention', 'xxmaj', 'george', 'xxmaj', 'clooney', ')', 'are', 'completely', 'wasted', 'in', 'this', 'nonsensical', 'movie', '.', '\n\n', 'xxmaj', 'the', 'director', '…', 'xxmaj', 'robe

In [20]:
num = Numericalize()
num.setup(toks)
nums = toks.map(num)
nums[0][:10]

TensorText([   2,    8,   62,   85,   38,   97,   56,  684, 2965,   27])

In [24]:
nums[0]

TensorText([   2,    8,   62,   85,   38,   97,   56,  684, 2965,   27,   24,  366, 1459,   51,   62,  500,  266,   49,   12,  980,  173,   73,   54,   77,   47,  139, 4488,   21,    9,   27,   85,   38,
         590,   73,   12, 1033, 7891,   25, 1853,   24,   37,   42,   14,  151,    5,  141,   55,    8,    9,  257, 2966, 5004,   10,   57,   47, 2176,   19,    9,   27,  134,   56,  442,  453,  247,
          14,   82,   10,   73,   13,  382, 1363,   50,   50,   73,   34,  196, 2487,   40, 2083,   87,   55,    8,    9,  111, 1777,   14,    9, 1460,   14,    8, 5678,    8, 1271,   13,    8, 4053,
           8, 3172,   39,   37,   15,  555,    8,  853,    8, 4054,   36,   41,  324,  528,   19,   20, 2639,   27,   10,   26,    8,    9,  173,   73,    8,  608,    8, 4489,   11,  685,   28,   45,
         107, 2177, 1364,  756, 2967, 6577,   11,    0,   11,  312,  652,   12,   82,   19,    8, 2640,   11,   13,    9,   83, 1365, 4055,  731,   73,   80, 1705,  142,  743,   29,    9,  102, 24

In [21]:
nums_dec = num.decode(nums[0][:10]); nums_dec

(#10) ['xxbos','xxmaj','what','could','have','been','an','excellent','hostage','movie']

In [22]:
tok.decode(nums_dec)

'xxbos xxmaj what could have been an excellent hostage movie'

In [26]:
tok((txts[0], txts[1]))

((#207) ['xxbos','xxmaj','what','could','have','been','an','excellent','hostage','movie'...],
 (#528) ['xxbos','xxmaj','one','of','the','problems','with','popular','culture',','...])

### Writing Your Own Transform

In [41]:
def f(x:int): return x*1
tfm = Transform(f)
tfm(2),tfm(2.0)

(2, 2.0)

In [43]:
@Transform
def f(x:int): return x*1
f(2),f(2.0)

(2, 2.0)

In [44]:
tfm("Hello")

'Hello'

In [45]:
f("Hello")

'Hello'

In [37]:
def func(x:int):
    return x *2

In [39]:
func("Hello")

'HelloHello'

In [46]:
class NormalizeMean(Transform):
    def setups(self, items): self.mean = sum(items)/len(items)
    def encodes(self, x): return x-self.mean
    def decodes(self, x): return x+self.mean

In [ ]:
tfm = NormalizeMean()
tfm.setup([1,2,3,4,5])
start = 2
y = tfm(start)
z = tfm.decode(y)
tfm.mean,y,z

In [50]:
np.mean([1,2,3,4,5,6])

3.5

In [54]:
normalize_transform = NormalizeMean()
normalize_transform.setup([1,2,3,4,5,6])
start = 2

y = normalize_transform(start) # bascially encodes method

In [55]:
y # 2-3.5

-1.5

In [56]:
z = normalize_transform.decode(y)

In [57]:
z

2.0

In [58]:
normalize_transform.mean

3.5

### Pipeline

In [61]:
text_tfms = Pipeline([tok, num])
t = text_tfms(txts[0]); t[:20]

TensorText([   2,    8,   62,   85,   38,   97,   56,  684, 2965,   27,   24,  366, 1459,   51,   62,  500,  266,   49,   12,  980])

In [62]:
text_tfms.decode(t)[:100]

'xxbos xxmaj what could have been an excellent hostage movie was totally ruined by what apparently lo'

<kbd>Ctrl</kbd> + <kbd>Shift</kbd> + <kbd>-</kbd> will split the current cell into two from where your cursor is.

## TfmdLists and Datasets: Transformed Collections

### TfmdLists

In [ ]:
tls = TfmdLists(files, [Tokenizer.from_folder(path), Numericalize])

In [ ]:
t = tls[0]; t[:20]

In [ ]:
tls.decode(t)[:100]

In [ ]:
tls.show(t)

In [ ]:
cut = int(len(files)*0.8)
splits = [list(range(cut)), list(range(cut,len(files)))]
tls = TfmdLists(files, [Tokenizer.from_folder(path), Numericalize], 
                splits=splits)

In [ ]:
tls.valid[0][:20]

In [ ]:
lbls = files.map(parent_label)
lbls

In [ ]:
cat = Categorize()
cat.setup(lbls)
cat.vocab, cat(lbls[0])

In [ ]:
tls_y = TfmdLists(files, [parent_label, Categorize()])
tls_y[0]

### Datasets

In [ ]:
x_tfms = [Tokenizer.from_folder(path), Numericalize]
y_tfms = [parent_label, Categorize()]
dsets = Datasets(files, [x_tfms, y_tfms])
x,y = dsets[0]
x[:20],y

In [ ]:
x_tfms = [Tokenizer.from_folder(path), Numericalize]
y_tfms = [parent_label, Categorize()]
dsets = Datasets(files, [x_tfms, y_tfms], splits=splits)
x,y = dsets.valid[0]
x[:20],y

In [ ]:
t = dsets.valid[0]
dsets.decode(t)

In [ ]:
dls = dsets.dataloaders(bs=64, before_batch=pad_input)

In [ ]:
tfms = [[Tokenizer.from_folder(path), Numericalize], [parent_label, Categorize]]
files = get_text_files(path, folders = ['train', 'test'])
splits = GrandparentSplitter(valid_name='test')(files)
dsets = Datasets(files, tfms, splits=splits)
dls = dsets.dataloaders(dl_type=SortedDL, before_batch=pad_input)

In [ ]:
path = untar_data(URLs.IMDB)
dls = DataBlock(
    blocks=(TextBlock.from_folder(path),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path)

## Applying the Mid-Level Data API: SiamesePair

In [ ]:
from fastai.vision.all import *
path = untar_data(URLs.PETS)
files = get_image_files(path/"images")

In [ ]:
class SiameseImage(fastuple):
    def show(self, ctx=None, **kwargs): 
        img1,img2,same_breed = self
        if not isinstance(img1, Tensor):
            if img2.size != img1.size: img2 = img2.resize(img1.size)
            t1,t2 = tensor(img1),tensor(img2)
            t1,t2 = t1.permute(2,0,1),t2.permute(2,0,1)
        else: t1,t2 = img1,img2
        line = t1.new_zeros(t1.shape[0], t1.shape[1], 10)
        return show_image(torch.cat([t1,line,t2], dim=2), 
                          title=same_breed, ctx=ctx)

In [ ]:
img = PILImage.create(files[0])
s = SiameseImage(img, img, True)
s.show();

In [ ]:
img1 = PILImage.create(files[1])
s1 = SiameseImage(img, img1, False)
s1.show();

In [ ]:
s2 = Resize(224)(s1)
s2.show();

In [ ]:
def label_func(fname):
    return re.match(r'^(.*)_\d+.jpg$', fname.name).groups()[0]

In [ ]:
class SiameseTransform(Transform):
    def __init__(self, files, label_func, splits):
        self.labels = files.map(label_func).unique()
        self.lbl2files = {l: L(f for f in files if label_func(f) == l) 
                          for l in self.labels}
        self.label_func = label_func
        self.valid = {f: self._draw(f) for f in files[splits[1]]}
        
    def encodes(self, f):
        f2,t = self.valid.get(f, self._draw(f))
        img1,img2 = PILImage.create(f),PILImage.create(f2)
        return SiameseImage(img1, img2, t)
    
    def _draw(self, f):
        same = random.random() < 0.5
        cls = self.label_func(f)
        if not same: 
            cls = random.choice(L(l for l in self.labels if l != cls)) 
        return random.choice(self.lbl2files[cls]),same

In [ ]:
splits = RandomSplitter()(files)
tfm = SiameseTransform(files, label_func, splits)
tfm(files[0]).show();

In [ ]:
tls = TfmdLists(files, tfm, splits=splits)
show_at(tls.valid, 0);

In [ ]:
dls = tls.dataloaders(after_item=[Resize(224), ToTensor], 
    after_batch=[IntToFloatTensor, Normalize.from_stats(*imagenet_stats)])

## Conclusion

## Questionnaire

1. Why do we say that fastai has a "layered" API? What does it mean?
1. Why does a `Transform` have a `decode` method? What does it do?
1. Why does a `Transform` have a `setup` method? What does it do?
1. How does a `Transform` work when called on a tuple?
1. Which methods do you need to implement when writing your own `Transform`?
1. Write a `Normalize` transform that fully normalizes items (subtract the mean and divide by the standard deviation of the dataset), and that can decode that behavior. Try not to peek!
1. Write a `Transform` that does the numericalization of tokenized texts (it should set its vocab automatically from the dataset seen and have a `decode` method). Look at the source code of fastai if you need help.
1. What is a `Pipeline`?
1. What is a `TfmdLists`? 
1. What is a `Datasets`? How is it different from a `TfmdLists`?
1. Why are `TfmdLists` and `Datasets` named with an "s"?
1. How can you build a `DataLoaders` from a `TfmdLists` or a `Datasets`?
1. How do you pass `item_tfms` and `batch_tfms` when building a `DataLoaders` from a `TfmdLists` or a `Datasets`?
1. What do you need to do when you want to have your custom items work with methods like `show_batch` or `show_results`?
1. Why can we easily apply fastai data augmentation transforms to the `SiamesePair` we built?

### Further Research

1. Use the mid-level API to prepare the data in `DataLoaders` on your own datasets. Try this with the Pet dataset and the Adult dataset from Chapter 1.
1. Look at the Siamese tutorial in the fastai documentation to learn how to customize the behavior of `show_batch` and `show_results` for new type of items. Implement it in your own project.

## Understanding fastai's Applications: Wrap Up

Congratulations—you've completed all of the chapters in this book that cover the key practical parts of training models and using deep learning! You know how to use all of fastai's built-in applications, and how to customize them using the data block API and loss functions. You even know how to create a neural network from scratch, and train it! (And hopefully you now know some of the questions to ask to make sure your creations help improve society too.)

The knowledge you already have is enough to create full working prototypes of many types of neural network applications. More importantly, it will help you understand the capabilities and limitations of deep learning models, and how to design a system that's well adapted to them.

In the rest of this book we will be pulling apart those applications, piece by piece, to understand the foundations they are built on. This is important knowledge for a deep learning practitioner, because it is what allows you to inspect and debug models that you build and create new applications that are customized for your particular projects.